In [1]:
# Load the CSV data into a DataFrame

import pandas as pd
df = pd.read_csv('feature_data3.csv')

# Convert the DataFrame to a NumPy array (matrix)
df_data = df.values

df_data = pd.DataFrame(df_data)

print(df_data)

           0         1          2           3           4           5   \
0    0.324081  0.113919  25.257430  221.713679  114.437756  254.385534   
1    0.891822  0.182954  11.288263   61.700028   61.073726   74.330344   
2    0.720290  0.183258  34.497578  188.245972  178.474088  267.226496   
3    0.456011  0.197963  36.007521  181.890080  114.354711  213.114992   
4    0.470130  0.136238  11.214203   82.313115   50.159745   91.005494   
..        ...       ...        ...         ...         ...         ...   
124  0.882767  0.241570  41.884166  173.383039  297.544955  212.150890   
125  0.434268  0.222052  40.629555  182.973112  155.012903  140.292551   
126  0.728373  0.234325  59.607648  254.380739  374.833296  216.187419   
127  0.923890  0.250879  53.087549  211.606485  346.255686  232.137890   
128  0.285109  0.063644  29.698362  466.631411  236.304888  124.679589   

             6           7              8           9   ...             90  \
0    227.002203  257.841036  2409

In [2]:
# import csv of lable
df = pd.read_csv('/home/tianyu/Desktop/data_base/labels.csv')

df_label = df['label'].values
df_label_num = []

for item in df_label:
    if item == 'M':
        df_label_num.append(1)
    else :
        df_label_num.append(0)
        
print(df_label_num)


[1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1]


In [39]:
# random forest k fold for old data and then test data

import numpy as np
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
group_models = []
group_num = 0
feature_importance = np.zeros(100)
X1 = df_data.iloc[:100]
for i in range(2):
    kf = KFold(n_splits=5, random_state=i+1, shuffle=True)

    X = df_data.iloc[0:80]
    y = df_label_num

    X = np.array(X)
    y = np.array(y)

    for train_index, test_index in kf.split(X):

        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        clf = RandomForestClassifier(n_estimators=20, random_state=i+1)
        clf.fit(X_train, y_train)
        
        y_pred = clf.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        if accuracy >= 0.7:
            group_models.append(clf)
            group_num += 1
            
            importances = clf.feature_importances_
            for feature, importance in zip(X1.columns, importances):
                feature_importance[feature] += importance
    
    print(i)
    
print(group_num)
print(feature_importance)

0
1
4
[0.0630968  0.01787991 0.02484447 0.06768459 0.04989252 0.07851558
 0.02923878 0.0715172  0.02233688 0.05983121 0.03739193 0.07612074
 0.01164885 0.05899256 0.02476771 0.02909437 0.29125864 0.00701181
 0.03014031 0.0291668  0.01441693 0.0105782  0.01764609 0.01786497
 0.04070442 0.02864611 0.00446244 0.02767132 0.06775246 0.1120471
 0.00506094 0.01444119 0.02027739 0.01595359 0.02719345 0.03047843
 0.02446293 0.03794672 0.03428929 0.03004174 0.02993619 0.00643537
 0.02293338 0.05442618 0.14655937 0.07492076 0.03201839 0.02150263
 0.03275199 0.04322253 0.00857969 0.02660512 0.02981548 0.01689222
 0.01141354 0.0370962  0.01075307 0.03231683 0.01425039 0.08221818
 0.00706202 0.0418646  0.10048205 0.09602768 0.04810114 0.01567015
 0.0282848  0.02092432 0.02672723 0.01557885 0.03456521 0.12835834
 0.04735598 0.03840811 0.02486742 0.02193509 0.03052445 0.02686184
 0.1755261  0.04629175 0.0203778  0.00769501 0.04072601 0.09173489
 0.03019365 0.00768681 0.04177877 0.07893356 0.04833936 0

In [42]:
# get test result through test data

import numpy as np
from scipy import stats
from sklearn.metrics import accuracy_score

X_task = df_data.iloc[79:100]
result_list = []

for model in group_models:
    result = model.predict(X_task)
    result_list.append(result)
    print(result)

vote_result = stats.mode(result_list, axis=0).mode.flatten()

print(f'final result is : \n {vote_result}')
print(df_label_num[79:100])

accuracy = accuracy_score(df_label_num[79:100], vote_result)
print(accuracy)

[0 0 0 0 0 1 0 1 1 1 1 0 0 0 1 1 1 0 0 1 0]
[0 0 0 0 0 1 0 1 1 1 1 0 0 0 1 1 1 0 0 1 0]
[0 0 0 0 0 1 0 1 1 1 1 0 0 0 1 1 1 0 0 1 0]
[0 0 0 0 0 1 0 1 1 1 1 0 0 0 1 1 1 0 0 1 0]
final result is : 
 [0 0 0 0 0 1 0 1 1 1 1 0 0 0 1 1 1 0 0 1 0]
[0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1]
0.6190476190476191


/tmp/ipykernel_5194/1204235807.py:15: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  vote_result = stats.mode(result_list, axis=0).mode.flatten()


In [54]:
# random forest for old data

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X = df_data.iloc[0:100]
y = df_label_num


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=7)

clf = RandomForestClassifier(n_estimators=20, random_state=110)

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
print(y_pred)
print(y_test)
accuracy = accuracy_score(y_test, y_pred)
print(accuracy)

importances = clf.feature_importances_

for feature, importance in zip(X.columns, importances):
    print(f"Feature: {feature}, Importance: {importance}")

[1 0 0 0 0 1 1 0 1 1 0 1 1 1 1 1 0 1 1 1]
[1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0]
0.9
Feature: 0, Importance: 0.0008621982306192865
Feature: 1, Importance: 0.01122003003790211
Feature: 2, Importance: 0.000625
Feature: 3, Importance: 0.024319616285357934
Feature: 4, Importance: 0.0025641025641025654
Feature: 5, Importance: 0.026409772908036134
Feature: 6, Importance: 0.01907094517066484
Feature: 7, Importance: 0.005068553930397519
Feature: 8, Importance: 0.0
Feature: 9, Importance: 0.0012099213551119196
Feature: 10, Importance: 0.019262018971434765
Feature: 11, Importance: 0.010592508417899129
Feature: 12, Importance: 0.004464410502146354
Feature: 13, Importance: 0.0020764471565386895
Feature: 14, Importance: 0.002227791701475911
Feature: 15, Importance: 0.00456265271080086
Feature: 16, Importance: 0.014245014245014237
Feature: 17, Importance: 0.005888241691520386
Feature: 18, Importance: 0.0
Feature: 19, Importance: 0.013797424694829507
Feature: 20, Importance: 0.0